In [2]:
from ray import rllib, tune
import ray
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.policy.policy_map import PolicyMap
from ray.rllib.evaluation.episode import MultiAgentEpisode
from ray.tune.registry import register_env
#from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from src.jss_lite.custom_torch_models import DenseModel
from ray.rllib.models.catalog import ModelCatalog
import gym
from src.jss_lite.jss_lite import jss_lite
ModelCatalog.register_custom_model("dense_model", DenseModel)
from copy import deepcopy
import numpy as np
import os
ray.shutdown()
#curr_dir=(os.path.dirname(__file__))
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
instance_list=['/resources/jsp_instances/standard/la01.txt','/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt','/resources/jsp_instances/standard/la05.txt']
instance_list=[curr_dir + s for s in instance_list]
#instance_list=['resources/jsp_instances/standard/ft06.txt','resources/jsp_instances/standard/ft10.txt','resources/jsp_instances/standard/ft20.txt','resources/jsp_instances/standard/abz5.txt','resources/jsp_instances/standard/abz6.txt','resources/jsp_instances/standard/abz7.txt']
#instance_list=['resources/jsp_instances/standard/ft06.txt']

train_agent=True
instance_path='resources/jsp_instances/standard/ft06.txt'
restore_agent= False
num_episodes = 5
restore_path= '/Users/felix/sciebo/masterarbeit/progra/model-based_rl/published_checkpoints/checkpoints_az_tune/ft06/checkpoint_000100/checkpoint-100'
config = {
    "framework": "torch",
    "disable_env_checking":True,
    "num_workers"       : 6,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    "explore"           :True,
    #"horizon"           : 600,
    #"soft_horizon"      : True,
    "num_sgd_iter"      : 1,
    #"horizon"           : 100,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.0,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : False,
        "add_dirichlet_noise": False,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}

# def env_creator(env_config):
#     env = jss_lite(instance_path='resources/jsp_instances/standard/ft06.txt')
#     return env

from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances

def env_creator(config):
    #env = jssp_light_obs_wrapper(jss_lite(instance_path=instance_path))
    env=jssp_light_obs_wrapper_multi_instances(instances_list=instance_list)
    return env

ModelCatalog.register_custom_model("dense_model", DenseModel)    



# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)

agent = AlphaZeroTrainer( config=config, env='custom_jssp')

2022-10-21 10:40:18,451	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!


In [2]:

import time
if train_agent:
    # checkpoint_path = analysis.get_last_checkpoint() or args.checkpoint
    ## use string number to restore pre trained agent
    # nr_restore="10"
    #checkpoint_path=f'checkpoints_az/rllib_checkpoint{nr_restore}/checkpoint_{nr_restore.zfill(6)}/checkpoint-{nr_restore}'
    #agent.load_checkpoint("training_checkpoints/checkpoints_az_jsslite/checkpoint-5")
    #print("awd")
    #agent.restore("checkpoints_az/rllib_checkpoint1/checkpoint_000001/checkpoint-1")
    #agent.load_checkpoint("published_checkpoints/az_taxi/checkpoint-34")
    print("start training")
    for _ in range(0,num_episodes):
        t=time.time()
        agent.train()
        print(f"training iteration {_} finished after {time.time()-t} seconds")
        agent.save_checkpoint(f"training_checkpoints/checkpoints_az_jsslite_ft06")
    
if restore_agent:

    agent.load_checkpoint(restore_path)

start training
training iteration 0 finished after 55.884531021118164 seconds
training iteration 1 finished after 55.267685890197754 seconds
training iteration 2 finished after 56.28659987449646 seconds
training iteration 3 finished after 59.49310803413391 seconds
training iteration 4 finished after 58.09264802932739 seconds


In [1]:

config = {
    "framework": "torch",
    "disable_env_checking":True,
    "num_workers"       : 6,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    "explore"           :False,
    #"horizon"           : 600,
    #"soft_horizon"      : True,
    "num_sgd_iter"      : 1,
    #"horizon"           : 100,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.5,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : True,
        "add_dirichlet_noise": False,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}

agent = AlphaZeroTrainer( config=config, env='custom_jssp')
#if restore_agent:
#restore_path='training_checkpoints/checkpoints_az_jsslite/checkpoint-5'
restore_path='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/checkpoints_az_jsslite_ft06/checkpoint-5'
agent.load_checkpoint(restore_path)

#agent.load_checkpoint("training_checkpoints/checkpoints_az_jsslite/checkpoint-5")

import time
length_list=[]
reward_list=[]
for _ in range(1):
    policy = agent.get_policy(DEFAULT_POLICY_ID)
    action_list=[]
    env = env_creator("s")

    obs = env.reset()
    # env2 is copy for later going evaluation


    episode = MultiAgentEpisode(
        PolicyMap(0,0),
        lambda _, __: DEFAULT_POLICY_ID,
        lambda: None,
        lambda _: None,
        0,
    )

    episode.user_data['initial_state'] = env.get_state()

    done = False
    t=time.time()
    while not done:
        action, _, _ = policy.compute_single_action(obs, episode=episode)
        action_list.append(action)
        #print(action)
        #print(obs['action_mask'][action])
        obs, reward, done, _ = env.step(action)
        #print(obs)
        #time.sleep(0.1)
        episode.length += 1

    length_list.append(episode.length)
    reward_list.append(reward)
    print(f"instance: {env.env.instance} in time: {time.time()-t}s")
    print(reward)
    #env.render()
    #env.close()

NameError: name 'AlphaZeroTrainer' is not defined

In [ ]:
ray.shutdown()

tune.run(
    "contrib/AlphaZero",
    stop={"training_iteration": 10},
    max_failures=0,
    #restore='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/published_checkpoints/checkpoints_az_tune/checkpoint_000500/checkpoint-500',
    config={
        "env": 'custom_jssp',
        "num_workers": 4,
        "rollout_fragment_length": 50,
        "train_batch_size": 50,
        "sgd_minibatch_size": 32,
        "lr": 1e-4,
        "num_sgd_iter": 1,
        "mcts_config": {
            "puct_coefficient": 1.5,
            "num_simulations": 100,
            "temperature": 1.0,
            "dirichlet_epsilon": 0.20,
            "dirichlet_noise": 0.03,
            "argmax_tree_policy": False,
            "add_dirichlet_noise": False,
        },
        "ranked_rewards": {
            "enable": True,
        },
        "model": {
            "custom_model": "dense_model",
        },
        "evaluation_interval": 1,
        "evaluation_config": {
            "render_env": True,
            "mcts_config": {
                "argmax_tree_policy": True,
                "add_dirichlet_noise": False,
            },
        },
    },
)

In [ ]:
# env = env_creator("setting")
# config.update(
#     simple_optimizer=True,
#     num_workers=0,
#     train_batch_size=0,
#     rollout_fragment_length=0,
#     timesteps_per_iteration=0,
#     evaluation_interval=1,
#     # evaluation_num_workers=...,
#     # evaluation_config=dict(explore=False),
#     # evaluation_num_episodes=...,
# )

# results = tune.run(
#     agent,
#     config=config)

In [ ]:
#agent.load_checkpoint("training_checkpoints/checkpoints_az_jsslite/checkpoint-5")
for _ in range(10):
    import time
    length_list=[]
    reward_list=[]
    for _ in range(1):
        policy = agent.get_policy(DEFAULT_POLICY_ID)
        action_list=[]
        env = env_creator("s")

        obs = env.reset()
        # env2 is copy for later going evaluation
        env2=deepcopy(env)

        episode = MultiAgentEpisode(
            PolicyMap(0,0),
            lambda _, __: DEFAULT_POLICY_ID,
            lambda: None,
            lambda _: None,
            0,
        )

        episode.user_data['initial_state'] = env.get_state()

        done = False

        while not done:
            action, _, _ = policy.compute_single_action(obs, episode=episode)
            action_list.append(action)
            #print(action_dic[action])
            obs, reward, done, _ = env.step(action)
            #print(obs)
            #env.render(render_mode='human')
            #time.sleep(0.1)
            episode.length += 1

        length_list.append(episode.length)
        reward_list.append(reward)
        print(reward)
        env.render()
        #env.close()

In [ ]:
env.render()

In [ ]:
print(reward)

In [ ]:
print(env.env.current_timestep)


In [ ]:
print(env.env.done)

In [ ]:
print(env.env.production_list)

In [ ]:
obs_1=env.reset()
#print(obs_1)

env_1=env_creator("a")
env_2=env_creator("b")
state_1=env_1.reset()
state_2=env_2.reset()

for a in a_list:
    state_1, reward_1, done_1, info_1 = env_1.step(a)
    state_2, reward_2, done_2, info_2 = env_2.step(a)

    if np.array_equal(state_1['obs'],state_2['obs'])==False :
        print("error obs")
        print(state_1)
        print(state_2)
    if np.array_equal(state_1['action_mask'],state_2['action_mask'])==False :
        print("error mask")
        print(state_1)
        print(state_2)

In [ ]:
for i in range(5):
    
    agent.load_checkpoint('training_checkpoints/checkpoints_az_jsslite/checkpoint-'+str(i+1))
    policy = agent.get_policy(DEFAULT_POLICY_ID)
    action_list=[]
    env = env_creator("s")

    obs = env.reset()
    # env2 is copy for later going evaluation
    #env2=deepcopy(env)

    episode = MultiAgentEpisode(
        PolicyMap(0,0),
        lambda _, __: DEFAULT_POLICY_ID,
        lambda: None,
        lambda _: None,
        0,
    )

    episode.user_data['initial_state'] = env.get_state()

    done = False
    steps=0
    t=time.time()
    while not done:
        action, _, _ = policy.compute_single_action(obs, episode=episode)
        action_list.append(action)
        #print(action_dic[action])
        obs, reward, done, _ = env.step(action)
        #print(obs)
        #env.render(render_mode='human')
        #time.sleep(0.1)
        steps+=1
    print(f"checkpoint {i} got reward {reward} in {steps} steps and time: {time.time()-t}")

In [ ]:
env.render()

In [ ]:
print(env.env.job_tasklength_matrix)

In [ ]:
print(reward)
print(env.env.count_finished_tasks_job_matrix)
env.render()
env.render(y_bar="Machine",x_bar="Job")


In [ ]:
env.render()
#env.render(y_bar="Machine",x_bar="Job")
print(done)
print(reward)
print(env.invalid_actions)

In [ ]:
env.reset()
from random import randrange
a_list=[]
for _ in range(30):
    state=env.reset()
    done=False
    i=0
    reward=0
    #print(state['obs'].shape)
    #for i in range(150):
    while not done:
        i+=1
        legal_action=state['action_mask']
        #print(legal_action)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        a_list.append(action)
        #action=randrange(env.action_space.n/2)
        #print(action)
        token=True
        #print("before:")
        #env.render(start_count=1,x_bar="Job",y_bar="Machine")
        state, reward, done, info=env.step(action)
    #env.render()
    print(reward)
    
    if reward > -68:
        print(env.env.get_legal_actions("yy"))
        print(env.done)
        print(env.env.done)
        print(env.env.production_list)
        env.env.render()


In [ ]:
print(env.env.get_legal_actions("a"))
print(env.env.current_timestep)
print(env.env.current_machines_status)
print(env.env.processed_and_max_time_job_matrix)

In [ ]:
print(env.env.get_legal_actions("a"))
print(env.env.blocked_actions)

In [ ]:

env.render()

In [ ]:
print(env.env.blocked_actions)
print(env.env.get_legal_actions("obs"))


In [ ]:
x=[[1,2,None],[1,2,3]]
if any(x==None for x in x):
        print(True)
for row in x:
    if None in row:
        #print(True)
        pass

In [ ]:
print(env.env.current_machines_status)
print(env.env.get_legal_actions("stat"))

In [ ]:
env.render()